### Imports


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import LabelEncoder
from torchvision.models import mobilenet_v3_small
from tqdm import tqdm

from src.dataset_loaders import ISAdetectDataset
from src.transforms import GrayScaleImage

### Setup


In [12]:
# Specify the model
MODEL = mobilenet_v3_small

# Model hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 1

# Set to an integer to limit the dataset size. Useful for debugging.
MAX_FILES_PER_ISA = 8

### Helper functions


In [13]:
def get_device():
    """
    Returns 'cuda' if CUDA is available, else 'mps' if Apple Silicon GPU is available,
    otherwise 'cpu'.
    """
    device = None
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    print(f"Using device: {device}")
    return device

### Prepare


In [ ]:
device = get_device()

dataset = ISAdetectDataset(
    dataset_path="../../dataset/ISAdetect/ISAdetect_full_dataset",
    transform=GrayScaleImage(224, 224),
    file_byte_read_limit=224 * 224,
    per_architecture_limit=MAX_FILES_PER_ISA,
)


groups = list(map(lambda x: x["architecture"], dataset.labels))
target_feature = list(map(lambda x: x["endianness"], dataset.labels))

### Train and evaluate


In [ ]:
logo = LeaveOneGroupOut()
label_encoder = LabelEncoder()

fold = 1
all_fold_accuracies = []
for train_idx, test_idx in logo.split(
    X=range(len(dataset)), y=target_feature, groups=groups
):
    print(f"\n=== Fold {fold} – leaving out group '{groups[test_idx[0]]}' ===")
    fold += 1

    all_train_labels = [dataset.labels[i]["endianness"] for i in train_idx]
    label_encoder.fit(all_train_labels)

    train_dataset = Subset(dataset, train_idx)
    test_dataset = Subset(dataset, test_idx)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
    )
    test_loader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
    )

    model = MODEL(num_classes=2, weights=None)
    model.features[0][0] = nn.Conv2d(
        1, 16, kernel_size=3, stride=2, padding=1, bias=False
    )
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Train model
    model.train()
    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}:")

        for images, labels in tqdm(train_loader):
            images = images.to(device)

            encoded_labels = torch.from_numpy(
                label_encoder.transform(labels["endianness"])
            ).to(device)

            optimizer.zero_grad()
            predictions = model(images)
            loss = criterion(predictions, encoded_labels)
            loss.backward()
            optimizer.step()

    # Evaluate model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            encoded_labels = torch.from_numpy(
                label_encoder.transform(labels["endianness"])
            ).to(device)

            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == encoded_labels).sum().item()
            total += encoded_labels.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    all_fold_accuracies.append(accuracy)

### Evaluate


In [ ]:
# Print overall performance across folds
mean_acc = np.mean(all_fold_accuracies)
std_acc = np.std(all_fold_accuracies)
print(f"\nOverall LOGO CV Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")